In [8]:
import sys
import os
from BEMT_program.solver import Solver
import numpy as np
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
 
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

BEM_path = "example_OPTIMIZE/prop.ini"
s_base   = Solver(BEM_path)
sections = s_base.rotor.sections 
base_s = s_base.rotor.s
base_c = np.array(s_base.rotor.c)
base_r = np.array(s_base.rotor.r)
base_pitch = np.array(s_base.rotor.pitch)
num_basesec = len(sections)

with HiddenPrints():
    df, sections = s_base.run_sweep('v_inf', 1, 1, 1)

print(df["CT"])


0    0.127655
Name: CT, dtype: object


In [16]:
import numpy as np
from scipy.interpolate import interp1d

Num_sec  = 50
Num_cacl = 1

list_sections  = []
list_r         = []
list_chord     = []
list_pitch     = []

for i in range(Num_sec):
    list_sections.append(base_s[0])

min_r  = float(base_r[0])
max_r  = float(base_r[-1])
list_r = np.linspace(min_r,max_r,num = Num_sec)

base_r_numeric = np.array(base_r, dtype=float)
base_c_numeric = np.array(base_c, dtype=float)
base_pitch_numeric = np.array(base_pitch, dtype=float)

f = interp1d(base_r_numeric, base_c_numeric, kind='cubic')
list_chord = f(list_r)

f = interp1d(base_r_numeric, base_pitch_numeric, kind='cubic')
list_pitch = f(list_r)


In [17]:
import time

print(len(list_sections),list_r.shape)
s_new   = Solver(BEM_path, s1=list_sections, r1=list_r, c1 = list_chord, p1 = list_pitch)

list_time = []
st = time.time()
with HiddenPrints():
    for i in range(Num_cacl):
        df, sections = s_new.run_sweep('v_inf', 1, 1, 1)

        end1 = time.time()
        list_time.append(end1 - st)
        st = end1

print(df["CT"])
list_time  = np.array(list_time)
total_time = np.sum(list_time)
avg_time   = np.average(list_time)
std_time   = np.std(list_time)

print("total time is {:.4f}, average time = {:.4f}, std = {:.4f}".format(total_time, avg_time, std_time))


50 (50,)
0    0.131349
Name: CT, dtype: object
total time is 0.1795, average time = 0.1795, std = 0.0000


In [23]:
a = 0.1322
b = 0.131349
print(abs((b-a)/a))

0.006437216338880623
